<a href="https://colab.research.google.com/github/Milafreire/WoMakersCode_DataAnalyst/blob/main/analise_sentimentos_eleicoesEUA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from textblob import TextBlob
from wordcloud import WordCloud
import plotly.graph_objects as go
import plotly.express as px

In [8]:
df_trump = pd.read_csv('/content/hashtag_donaldtrump.csv', lineterminator='\n') 
df_biden = pd.read_csv('/content/hashtag_joebiden.csv', lineterminator='\n')

In [9]:
# renomeia para US tudo que for Estados Unidos
df_trump['country']=df_trump['country'].replace({"United States of America" : "US", "United States" : "US"})
df_biden['country']=df_biden['country'].replace({"United States of America" : "US", "United States" : "US"})

In [ ]:
print(df_trump.head())
print(df_biden.head())

In [11]:
textblob1 = TextBlob(df_trump["tweet"][1000])
print("Trump :",textblob1.sentiment)
textblob2 = TextBlob(df_biden["tweet"][1000])
print("Biden :",textblob2.sentiment)

Trump : Sentiment(polarity=-0.16666666666666666, subjectivity=0.43333333333333335)
Biden : Sentiment(polarity=-0.033333333333333326, subjectivity=0.6333333333333333)


In [31]:
def find_pol(review):
    return TextBlob(review).sentiment.polarity
df_trump["Sentiment Polarity"] = df_trump["tweet"].apply(find_pol)
print(df_trump.tail())

df_biden["Sentiment Polarity"] = df_biden["tweet"].apply(find_pol)
print(df_biden.tail())

                created_at      tweet_id  \
20058  2020-10-16 00:57:19  1.316906e+18   
20059  2020-10-16 00:57:19  1.316906e+18   
20060  2020-10-16 00:57:21  1.316906e+18   
20061  2020-10-16 00:57:21  1.316906e+18   
20062  2020-10-16 00:57:22  1.316906e+18   

                                                   tweet  likes  \
20058  @GOPChairwoman\n@PressSec\n@realDonaldTrump\n\...    0.0   
20059  Per usual @realDonaldTrump getting asked ridic...    1.0   
20060  Glad I’m watching the Biden town hall. I’ll ca...    3.0   
20061  Can someone please get that Black woman behind...    1.0   
20062  “I have to say you have a great smile.” 😂👏👏👏 P...    1.0   

       retweet_count               source       user_id  \
20058            0.0  Twitter for Android  2.855137e+09   
20059            1.0   Twitter for iPhone  2.198664e+08   
20060            2.0  Twitter for Android  7.880754e+17   
20061            0.0      Twitter Web App  7.727824e+07   
20062            0.0   Twitter for iP

TypeError: ignored

In [20]:
df_trump["Expression Label"] = np.where(df_trump["Sentiment Polarity"]>0, "positive", "negative")
df_trump["Expression Label"][df_trump["Sentiment Polarity"]==0]="Neutral"
print(df_trump.tail())

df_biden["Expression Label"] = np.where(df_biden["Sentiment Polarity"]>0, "positive", "negative")
df_biden["Expression Label"][df_biden["Sentiment Polarity"]==0]="Neutral"
print(df_biden.tail())

                created_at      tweet_id  \
20060  2020-10-16 00:57:21  1.316906e+18   
20061  2020-10-16 00:57:21  1.316906e+18   
20062  2020-10-16 00:57:22  1.316906e+18   
20063  2020-10-16 00:57:23  1.316906e+18   
20064  2020-10-16 00:57:25  1.316906e+18   

                                                   tweet  likes  \
20060  Glad I’m watching the Biden town hall. I’ll ca...    3.0   
20061  Can someone please get that Black woman behind...    1.0   
20062  “I have to say you have a great smile.” 😂👏👏👏 P...    1.0   
20063      THEY'RE THE SAME THING #Trump #DACA #Dreamers    1.0   
20064  Hey sis, nod your head if you need help! #Trum...   21.0   

       retweet_count               source       user_id  \
20060            2.0  Twitter for Android  7.880754e+17   
20061            0.0      Twitter Web App  7.727824e+07   
20062            0.0   Twitter for iPhone  6.252676e+08   
20063            0.0      Twitter Web App  3.649996e+08   
20064            NaN                 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


KeyError: ignored

In [30]:
reviews1 = df_trump[df_trump['Sentiment Polarity'] == 0.0000]
print(reviews1.shape)

cond1=df_trump['Sentiment Polarity'].isin(reviews1['Sentiment Polarity'])
df_trump.drop(df_trump[cond1].index, inplace = True)
print(df_trump.shape)

reviews2 = df_biden[df_biden['Sentiment Polarity'] == 0.0000]
print(reviews2.shape)

cond2=df_biden['Sentiment Polarity'].isin(reviews1['Sentiment Polarity'])
df_biden.drop(df_biden[cond2].index, inplace = True)
print(df_biden.shape)

(0, 23)
(11073, 23)


KeyError: ignored

In [22]:
# Donald Trump
np.random.seed(10)
remove_n =324
drop_indices = np.random.choice(df_trump.index, remove_n, replace=False)
df_subset_trump = df_trump.drop(drop_indices)
print(df_subset_trump.shape)
# Joe Biden
np.random.seed(10)
remove_n =324
drop_indices = np.random.choice(df_biden.index, remove_n, replace=False)
df_subset_biden = df_biden.drop(drop_indices)
print(df_subset_biden.shape)

(10749, 23)
(22383, 21)


In [29]:
count_1 = df_subset_trump.groupby('Expression Label').count()
print(count_1)

negative_per1 = (count_1['Sentiment Polarity'][0]/1000)*10
positive_per1 = (count_1['Sentiment Polarity'][1]/1000)*100

count_2 = df_subset_biden.groupby('Expression Label').count()
print(count_2)

negative_per2 = (count_2['Sentiment Polarity'][0]/1000)*100
positive_per2 = (count_2['Sentiment Polarity'][1]/1000)*100

Politicians = ['Joe Biden', 'Donald Trump']
lis_pos = [positive_per1, positive_per2]
lis_neg = [negative_per1, negative_per2]

fig = go.Figure(data=[
    go.Bar(name='Positive', x=Politicians, y=lis_pos),
    go.Bar(name='Negative', x=Politicians, y=lis_neg)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

                  created_at  tweet_id  tweet  likes  retweet_count  source  \
Expression Label                                                              
negative                4436      4436   4436   4436           4436    4436   
positive                6313      6313   6313   6313           6313    6269   

                  user_id  user_name  user_screen_name  user_description  ...  \
Expression Label                                                          ...   
negative             4436       4436              4436              4007  ...   
positive             6313       6312              6313              5678  ...   

                  user_location   lat  long  city  country  continent  state  \
Expression Label                                                               
negative                   3102  2306  2306  1091     2296       2296   1898   
positive                   4477  3394  3394  1658     3385       3385   2696   

                  state_code  collect

KeyError: ignored